<a href="https://colab.research.google.com/github/delanol/projetos/blob/main/C%C3%B3digo_do_Agente_de_Conex%C3%A3o_Consumidor_Solu%C3%A7%C3%A3o_Sustent%C3%A1vel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from IPython import get_ipython
from IPython.display import display
# %%
import google.generativeai as genai
import pandas as pd

# Configura a API Key do Google Gemini
import os
from google.colab import userdata
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

# Configura o cliente da SDK do Gemini
from google import genai
client = genai.Client()
MODEL_ID = "gemini-2.0-flash"

# Instalar Framework de agentes do Google
################################################
!pip install -q google-adk
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap
from IPython.display import display, Markdown
import requests
import warnings
warnings.filterwarnings("ignore")

# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])
    final_response = ""
    # Itera pelos eventos
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response

# 1. Agente de Perfil do Consumidor
def agente_perfil_consumidor():
    """
    Agente responsável por coletar informações sobre os interesses e valores de sustentabilidade do consumidor.
    """
    perfil_consumidor = Agent(
        name="agente_perfil_consumidor",
        model=MODEL_ID,
        instruction="Você é um especialista em entender o perfil de consumidores interessados em sustentabilidade. Faça perguntas ao usuário para descobrir seus interesses, valores e preferências em relação a produtos e serviços sustentáveis. Tente descobrir quais são suas principais preocupações ambientais, se ele verifica a origem e o impacto dos produtos, quais selos de sustentabilidade conhece, se está disposto a pagar mais por opções sustentáveis e quais setores de consumo são mais importantes para ele.",
        description="Coleta informações sobre o perfil de sustentabilidade do consumidor.",
    )
    return perfil_consumidor

# 2. Agente de Busca de Soluções Sustentáveis
def agente_busca_solucoes(conjunto_de_dados: pd.DataFrame):
    """
    Agente responsável por buscar empresas, projetos e produtos/serviços que correspondam ao perfil do consumidor.

    Args:
        conjunto_de_dados (pd.DataFrame): DataFrame contendo informações sobre soluções sustentáveis.
    """
    busca_solucoes = Agent(
        name="agente_busca_solucoes",
        model=MODEL_ID,
        instruction="Você é um especialista em encontrar soluções sustentáveis. Use o perfil do consumidor fornecido para buscar em um banco de dados as empresas, projetos e produtos/serviços mais relevantes. Considere as preferências do consumidor em relação a setores de consumo, selos de sustentabilidade, disposição a pagar mais e preocupações ambientais.",
        description="Busca soluções sustentáveis com base no perfil do consumidor.",
    )
    # Armazena o conjunto de dados no agente - REMOVIDO
    # busca_solucoes.conjunto_de_dados = conjunto_de_dados
    return busca_solucoes

# 3. Agente de Apresentação Personalizada
def agente_apresentacao():
    """
    Agente responsável por apresentar as soluções sustentáveis de forma personalizada.
    """
    apresentacao_personalizada = Agent(
        name="agente_apresentacao",
        model=MODEL_ID,
        instruction="Você é um especialista em comunicação persuasiva. Apresente as soluções sustentáveis para o consumidor de forma personalizada, destacando os aspectos mais relevantes de acordo com seu perfil. Use uma linguagem clara, objetiva e persuasiva. Inclua informações detalhadas sobre certificações, práticas de produção, impacto ambiental, benefícios sociais e econômicos, preço e disponibilidade, e avaliações de outros consumidores.",
        description="Apresenta soluções sustentáveis de forma personalizada.",
    )
    return apresentacao_personalizada

# Função principal para orquestrar a interação
def interacao_consumidor_sustentavel(conjunto_de_dados: pd.DataFrame):
    """
    Função que coordena a interação entre os agentes para conectar consumidores com soluções sustentáveis.

    Args:
        conjunto_de_dados (pd.DataFrame): DataFrame contendo informações sobre soluções sustentáveis.
    """
    # Cria os agentes
    agente_perfil = agente_perfil_consumidor()
    # Não passamos mais o DataFrame aqui, ele será passado no prompt
    agente_busca = agente_busca_solucoes(None)
    agente_apresenta = agente_apresentacao()

    # Inicia a conversa com o consumidor
    pergunta_inicial = "Olá! Sou seu assistente para encontrar soluções sustentáveis. Para começar, poderia me dizer quais são suas principais preocupações ambientais?"
    resposta_perfil = call_agent(agente_perfil, pergunta_inicial)
    print(resposta_perfil)  # Exibe a resposta do agente de perfil

    # Coleta mais informações para o perfil (adapte conforme necessário)
    perguntas_perfil = [
        "Você costuma verificar a origem e o impacto ambiental dos produtos que compra?",
        "Quais selos ou certificações de sustentabilidade você conhece ou valoriza?",
        "Você está disposto a pagar um pouco mais por produtos/serviços mais sustentáveis?",
        "Quais setores de consumo são mais importantes para você em termos de sustentabilidade? (alimentação, moda, transporte, energia, etc.)"
    ]
    for pergunta in perguntas_perfil:
        resposta_perfil = call_agent(agente_perfil, pergunta)
        print(resposta_perfil)

    # Formata o conjunto de dados como uma string para incluir no prompt
    # Você pode ajustar a formatação conforme necessário para o agente entender
    dados_formatados = conjunto_de_dados.to_string() # Ou to_json(), to_dict(), etc.

    # Busca soluções sustentáveis com base no perfil e nos dados fornecidos no prompt
    busca_prompt = f"""
    Aqui está o perfil do consumidor:
    {resposta_perfil}

    Aqui estão os dados de soluções sustentáveis:
    {dados_formatados}

    Com base no perfil do consumidor, encontre as soluções sustentáveis mais relevantes a partir dos dados fornecidos.
    """
    resultados_busca = call_agent(agente_busca, busca_prompt)
    print("Resultados da Busca:\n", resultados_busca) #debug

    # Apresenta as soluções ao consumidor
    resposta_apresentacao = call_agent(agente_apresenta, resultados_busca + "\n" + resposta_perfil) #passa resultados da busca e resposta do perfil
    print(resposta_apresentacao)

if __name__ == "__main__":
    # Exemplo de uso com um DataFrame de exemplo (substitua com seus dados reais)
    data = {
        'Nome': ['Produto A', 'Produto B', 'Serviço X', 'Serviço Y'],
        'Descrição': [
            'Produto com selo ecológico, feito de materiais reciclados',
            'Produto de comércio justo, produzido por cooperativas',
            'Serviço de entrega de bicicleta',
            'Consultoria para empresas em sustentabilidade'
        ],
        'Setor': ['Alimentos', 'Moda', 'Transporte', 'Serviços'],
        'Certificações': ['Selo Ecológico XYZ', 'Comércio Justo ABC', None, None],
        'Preco': [25.00, 30.00, 15.00, 100.00],
        'Avaliacao': [4.5, 4.8, 4.2, 4.9]
    }
    df_solucoes = pd.DataFrame(data)

    interacao_consumidor_sustentavel(df_solucoes)

Olá! Que bom ter você aqui para me ajudar a encontrar soluções sustentáveis.

Minhas principais preocupações ambientais são:

1.  **Mudanças climáticas:** Aumento da temperatura global, eventos climáticos extremos e seus impactos na vida humana e nos ecossistemas.
2.  **Poluição plástica:** A quantidade absurda de plástico nos oceanos e seus efeitos na vida marinha, além da contaminação do solo e da água.
3.  **Desmatamento:** A perda de florestas, especialmente na Amazônia, e seus impactos na biodiversidade, no clima e nas comunidades locais.
4.  **Consumo excessivo:** A cultura do consumo desenfreado e a produção de lixo, que esgotam os recursos naturais e geram poluição.
5.  **Crise hídrica:** A escassez de água potável e a má gestão dos recursos hídricos.

Essas são as questões que mais me preocupam e para as quais busco soluções sustentáveis no meu dia a dia.


Sim, tento ao máximo. Para entender melhor como isso se aplica às suas escolhas, você poderia me dizer:

1.  **Quais são 